# Classification de la question

## 1. Prétraitement textuel de la base de données

### A. Préparation du fichier

In [1]:
import pandas as pd

In [2]:
df_faq = pd.read_pickle('faq_centerPark.pkl')
df_faq.head()

,question,reponse,theme
0,Quels sont les jours d’arrivée ?,Il est possible d’arriver n’importe quel jour ...,Préparer mon séjour
1,Comment évaluer le confort de mon domaine et d...,Le classement par « birdies » évalue l’offre C...,Préparer mon séjour
2,Quels sont les services et activités compris d...,"En réservant votre hébergement, vous bénéficie...",Préparer mon séjour
3,Comment réserver mes activités ?,"Lors de la réservation de votre hébergement, v...",Préparer mon séjour
4,Où trouver le plan du domaine ?,"Sur la page d'accueil de notre site, cliquez s...",Préparer mon séjour


In [3]:
df_faq.shape

(54, 3)

Notre jeu de données est composé de :
    - 54 lignes
    - 3 colonnes
    - 5 thèmes différents
On pose :
    - 1 = Préparer mon séjour
    - 2 = Réserver et payer
    - 3 = Gérer ma réservation
    - 4 = Mon séjour
    - 5 = Assurances

In [5]:
df_faq.to_csv('jeu_test_similarite.csv', sep=";")

In [6]:
dic_code_theme = {"Préparer mon séjour": 1,
                  "Réserver et payer": 2,
                  "Gérer ma réservation": 3,
                  "Mon séjour": 4,
                  "Assurances": 5}
dic_decode_theme = {val: key for key, val in dic_code_theme.items()}

In [7]:
df_faq["theme"].replace(dic_code_theme, inplace=True)

In [8]:
df_question = df_faq[['question', 'theme']]
df_question.rename(columns={'question': 'texte', 'theme': 'theme'}, inplace=True)
df_reponse = df_faq[['reponse', 'theme']]
df_reponse.rename(columns={'reponse': 'texte', 'theme': 'theme'}, inplace=True)

C:\Users\enora\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [9]:
df_concat = pd.concat([df_question, df_reponse])

In [10]:
# Sauvegarde
df_concat.to_pickle('df_concat.pkl')

### B. Nettoyage du texte

In [11]:
import os
import numpy as np
import re
from collections import Counter

# pour le nettoyage du texte
import nltk
import string
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#pour la classification
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.svm import SVC

import unidecode

In [12]:
from sklearn.model_selection import train_test_split

df_concat = pd.read_pickle('df_concat.pkl')
X_train, X_test, y_train, y_test = train_test_split(df_concat['texte'], 
                                                    df_concat['theme'],
                                                    train_size=0.7,
                                                    random_state=5)

In [13]:
stemmer = SnowballStemmer('french')
#tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) # tokenizer for tweet
nlp = spacy.load('fr_core_news_sm')

sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
sw.sort()

def lemmatise_text(text):
    lst_lematised = [token.lemma_ for token in nlp(text)]
    return ' '.join(lst_lematised).lower()


def stem_text(text):
    #lst_stemmerised = [stemmer.stem(token) for token in tokenizer.tokenize(text)] 
    lst_stemmerised = [stemmer.stem(token) for token in word_tokenize(text)]
    return ' '.join(lst_stemmerised)


def replace_words_with_pos_tag(text):
    lst_tags = [token.pos_ for token in nlp(text)]
    return ' '.join(lst_tags)


def ner(text): #entites nommees
    dico_remplacement = {entite_nommee.text : entite_nommee.label_ for entite_nommee in nlp(text).ents}
    for entite_nommee, remplacement in dico_remplacement.items():
        text = text.replace(entite_nommee, remplacement)
    return text


def substitute_punctuation(text):
    return ' '.join(text.replace("'", ' ').translate(str.maketrans('', '', string.punctuation)).split())


def supp(text):
    return text.replace("«", "").replace("’", "").replace("•", "").replace("®", "")


def supprime_accent(txt):
    return unidecode.unidecode(txt)

In [14]:
X_train_clean = (X_train.apply(lemmatise_text)
                        .apply(stem_text)
                        .apply(substitute_punctuation)
                        .apply(supp)
                )

X_test_clean = (X_test.apply(lemmatise_text)
                      .apply(stem_text)
                      .apply(substitute_punctuation)
                      .apply(supp)
               )

## C. Test de différents modèles

##### a. Les différents vectoriseurs

In [15]:
#vectoriseur binaire

bin_count = CountVectorizer(binary=True)
bin_count.fit(X_train_clean)
X_train_clean_vectorized_bin = bin_count.transform(X_train_clean)
X_test_clean_vectorized_bin = bin_count.transform(X_test_clean)

In [16]:
#vectoriseur numérique discret

vect_count = CountVectorizer(binary=False)
vect_count.fit(X_train_clean)
X_train_clean_vectorized_count = vect_count.transform(X_train_clean)
X_test_clean_vectorized_count = vect_count.transform(X_test_clean)

In [17]:
#vectoriseur numérique continu : TF-IDF

vect_tfidf = TfidfVectorizer(stop_words=sw)
vect_tfidf.fit(X_train_clean)
X_train_clean_vectorized_tfidf = vect_tfidf.transform(X_train_clean)
X_test_clean_vectorized_tfidf = vect_tfidf.transform(X_test_clean) 

## export vectorizer
from joblib import dump
dump(vect_tfidf, 'vectorizer_classif_theme.joblib')

['vectorizer_classif_theme.joblib']

#### b. Les différents modèles

Nous entraînerons des modèles de classification appartenant à quelques familles d'algorithmes d'apprentissage automatique classique. L'objectif est de comparer non seulement les performances des différentes méthodes entre elles, mais aussi la performance d'une même méthode sur des représentations différentes du texte.

##### b.1 DummyClassifier

In [18]:
from sklearn.dummy import DummyClassifier

In [19]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_clean_vectorized_count, y_train)
predictions_valid = random_uniform.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.06060606060606061

In [20]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = random_uniform.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.12121212121212122

In [21]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = random_uniform.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.21212121212121213

##### b.2 Classifieur naïf bayesien

In [22]:
from sklearn.naive_bayes import MultinomialNB

In [23]:
model_nb = MultinomialNB().fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_nb.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.5757575757575758

In [24]:
model_nb = MultinomialNB().fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_nb.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.6060606060606061

In [25]:
model_nb = MultinomialNB().fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_nb.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.5454545454545454

#### b.3 Complement NB

In [26]:
from sklearn.naive_bayes import ComplementNB

In [27]:
model_comp = ComplementNB().fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_comp.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.5454545454545454

In [28]:
model_comp = ComplementNB().fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_comp.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.6060606060606061

In [29]:
model_comp = ComplementNB().fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_comp.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.6666666666666666

#### b.4 BernoulliNB

In [30]:
from sklearn.naive_bayes import BernoulliNB

In [31]:
model_bern = BernoulliNB().fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_bern.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.36363636363636365

In [32]:
model_bern = BernoulliNB().fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_bern.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.36363636363636365

In [33]:
model_bern = BernoulliNB().fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_bern.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.36363636363636365

#### b.5KNeighborsClassifier

In [34]:
from sklearn.neighbors import KNeighborsClassifier

In [35]:
model_knn = KNeighborsClassifier(4).fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_knn.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.2727272727272727

In [36]:
model_knn = KNeighborsClassifier(4).fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_knn.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.30303030303030304

In [37]:
model_knn = KNeighborsClassifier(4).fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_knn.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.6363636363636364

#### b.7. SVM

In [38]:
from sklearn.svm import SVC

In [39]:
model_svm = SVC(kernel='linear', C=0.9).fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.3939393939393939

In [40]:
model_svm = SVC(kernel='linear', C=0.9).fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.3939393939393939

In [41]:
model_svm = SVC(kernel='linear', C=0.9).fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.5757575757575758

#### b.8 Réseaux de neurones de convolution

In [42]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, SpatialDropout1D, MaxPooling1D, Conv1D, Flatten, MaxPooling2D, Conv2D

Using TensorFlow backend.


In [43]:
XX_train = X_train_clean_vectorized_tfidf.toarray().reshape(X_train_clean_vectorized_tfidf.shape[0],1,
                                                X_train_clean_vectorized_tfidf.shape[1])
XX_test = X_test_clean_vectorized_tfidf.toarray().reshape(X_test_clean_vectorized_tfidf.shape[0],1,
                                                X_test_clean_vectorized_tfidf.shape[1])
YY_train = pd.get_dummies(y_train)
YY_test = pd.get_dummies(y_test)

In [49]:
n_timesteps = 1
n_features = XX_train.shape[2]

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(units=YY_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

epochs = 150
batch_size = 32
history = model.fit(XX_train, YY_train, epochs=epochs, batch_size=batch_size, validation_split=0.4)

Train on 45 samples, validate on 30 samples
Epoch 1/150
45/45 [==============================] - ETA: 0s - loss: 1.6218 - accuracy: 0.12 - 0s 5ms/step - loss: 1.6177 - accuracy: 0.1556 - val_loss: 1.6150 - val_accuracy: 0.1333
Epoch 2/150
45/45 [==============================] - ETA: 0s - loss: 1.6106 - accuracy: 0.28 - 0s 600us/step - loss: 1.6069 - accuracy: 0.2667 - val_loss: 1.6119 - val_accuracy: 0.2000
Epoch 3/150
45/45 [==============================] - ETA: 0s - loss: 1.6006 - accuracy: 0.31 - 0s 667us/step - loss: 1.5989 - accuracy: 0.3333 - val_loss: 1.6093 - val_accuracy: 0.2000
Epoch 4/150
45/45 [==============================] - ETA: 0s - loss: 1.5986 - accuracy: 0.37 - 0s 511us/step - loss: 1.5911 - accuracy: 0.4444 - val_loss: 1.6067 - val_accuracy: 0.3000
Epoch 5/150
45/45 [==============================] - ETA: 0s - loss: 1.5803 - accuracy: 0.50 - 0s 555us/step - loss: 1.5837 - accuracy: 0.4667 - val_loss: 1.6044 - val_accuracy: 0.3333
Epoch 6/150
45/45 [==============

Epoch 45/150
45/45 [==============================] - ETA: 0s - loss: 0.9349 - accuracy: 0.90 - 0s 378us/step - loss: 0.9384 - accuracy: 0.9333 - val_loss: 1.3925 - val_accuracy: 0.4333
Epoch 46/150
45/45 [==============================] - ETA: 0s - loss: 0.9181 - accuracy: 0.93 - 0s 578us/step - loss: 0.9134 - accuracy: 0.9333 - val_loss: 1.3833 - val_accuracy: 0.4333
Epoch 47/150
45/45 [==============================] - ETA: 0s - loss: 0.8999 - accuracy: 0.93 - 0s 511us/step - loss: 0.8875 - accuracy: 0.9333 - val_loss: 1.3733 - val_accuracy: 0.4333
Epoch 48/150
45/45 [==============================] - ETA: 0s - loss: 0.8467 - accuracy: 0.90 - 0s 578us/step - loss: 0.8625 - accuracy: 0.9333 - val_loss: 1.3630 - val_accuracy: 0.4333
Epoch 49/150
45/45 [==============================] - ETA: 0s - loss: 0.8369 - accuracy: 0.93 - 0s 400us/step - loss: 0.8372 - accuracy: 0.9333 - val_loss: 1.3535 - val_accuracy: 0.4333
Epoch 50/150
45/45 [==============================] - ETA: 0s - loss: 

Epoch 133/150
45/45 [==============================] - ETA: 0s - loss: 0.0615 - accuracy: 1.00 - 0s 444us/step - loss: 0.0713 - accuracy: 0.9778 - val_loss: 0.8976 - val_accuracy: 0.6333
Epoch 134/150
45/45 [==============================] - ETA: 0s - loss: 0.0611 - accuracy: 1.00 - 0s 422us/step - loss: 0.0697 - accuracy: 0.9778 - val_loss: 0.9016 - val_accuracy: 0.6333
Epoch 135/150
45/45 [==============================] - ETA: 0s - loss: 0.0806 - accuracy: 0.96 - 0s 489us/step - loss: 0.0696 - accuracy: 0.9778 - val_loss: 0.9065 - val_accuracy: 0.6000
Epoch 136/150
45/45 [==============================] - ETA: 0s - loss: 0.0774 - accuracy: 0.96 - 0s 400us/step - loss: 0.0694 - accuracy: 0.9778 - val_loss: 0.9092 - val_accuracy: 0.6000
Epoch 137/150
45/45 [==============================] - ETA: 0s - loss: 0.0834 - accuracy: 0.96 - 0s 555us/step - loss: 0.0694 - accuracy: 0.9778 - val_loss: 0.9102 - val_accuracy: 0.6000
Epoch 138/150
45/45 [==============================] - ETA: 0s - 

In [50]:
pred_proba=model.predict(XX_test)
idx = np.argmax(pred_proba, axis=-1)
YY_pred = np.zeros( pred_proba.shape )
YY_pred[ np.arange(YY_pred.shape[0]), idx] = 1
accuracy_score(YY_test, YY_pred)

0.5757575757575758

In [46]:
from joblib import dump
dump(model, 'model_classif_theme.joblib')

['model_classif_theme.joblib']